# Neural Shrub - Leaves

In [1]:
import time
import os, sys
import string

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras import Sequential
from keras import layers

import pandas as pd
import numpy as np

/home/shashwati/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data(dataset):
    if os.path.isfile(dataset):
        print("Loading ", dataset, " dataset ...")
        data = pd.read_csv(dataset)
        print("\nDataset loaded successfully\n\n")
        return data
    else:
        print('File not found')
        print('\n\nExiting...')
        sys.exit()

In [3]:
#The column names are [a, b, c, ..., z, A, B, C, ..., W]
columnNames = list(string.ascii_lowercase) \
              + list(string.ascii_uppercase)[:23]

In [4]:
def get_data():   
    train_dataset = load_data('./sensIT_train.csv')
    label_train = train_dataset['result']
    train = train_dataset[columnNames]

    test_dataset = load_data('./sensIT_test.csv')
    label_test = test_dataset['result']
    test = test_dataset[columnNames]
    
    return train, label_train, test, label_test

In [5]:
from sklearn.tree._tree import TREE_LEAF

def prune_index(inner_tree, index, threshold):
    if inner_tree.value[index].min() < threshold:
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF
    # if there are shildren, visit them as well
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_index(inner_tree, inner_tree.children_left[index], threshold)
        prune_index(inner_tree, inner_tree.children_right[index], threshold)

In [6]:
# Makes the decision tree
def decision_tree(train, label):
    dt = DecisionTreeClassifier(max_depth = 8, min_samples_leaf=500, random_state = 1)
    dt.fit(train, label)
    prune_index(dt.tree_, 0, 5)
    return dt

In [7]:
# Class_data: list of instances belonging to a class
# Each instance consists of the predictor_values and the actual class
def neural_network(class_data):
    nn_train = []
    nn_label = []
    
    for instance in class_data:
        nn_train.append(instance[0]) # predictor
        nn_label.append(instance[1]) # actual class
    
    nn_train = np.array(nn_train)
    nn_label = np.array(nn_label)

    # Preprocessing
    encoder = LabelEncoder()
    encoder.fit(nn_label)
    nn_label = encoder.transform(nn_label)
    nn_label = np_utils.to_categorical(nn_label)
    
    print(nn_train.shape, nn_label.shape)
        
    out = nn_label.shape
    print(out[1])
    exit()
    # Neural network structure
    model = Sequential()
    model.add(layers.Dense(30,init = 'uniform', activation = 'relu', input_dim = 49))
    model.add(layers.Dense(10,init = 'uniform', activation = 'relu'))
    model.add(layers.Dense(out[1], init = 'uniform', activation = 'softmax'))

    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    model.fit(nn_train, nn_label, epochs=15, batch_size=500)
    
    return model

In [8]:
# builds the neural shrub
def neural_shrubs(tree, train, label):
    train = np.array(train)
    label = np.array(label)
  
    # leave_id: index of the leaf that cantains the instance
    leave_id = tree.apply(train) 

    classes = dict()

    for x in range(len(train)):
        leaf = leave_id[x]
        
        # Gets the class for each leaf
        #.value: contains value of all the tree nodes
        #.value[leaf]: returns the value of the leaf
        #idx = tree.tree_.value[leaf][0][0]
        
        # insert the instance into the class
        if leaf in classes.keys():
            classes[leaf].append([train[x], label[x]])
        else:
            classes[leaf] = [[train[x], label[x]]]
    
    # stores the neural network for each class
    nn_models = dict()
    
    #stores the max time taken to build a neural network
    max_time = 0;

    for key in classes.keys():
        
        start = time.time()
        model = neural_network(classes[key])
        end = time.time()
        
        time_taken = end - start
        if max_time < time_taken:
            max_time = time_taken
        
        nn_models[key] = model

    # returns a neural network for each class and the max 
    # time taken to build the neural network
    return nn_models, max_time

In [9]:
# The algorithm to build the neural shrub
train, train_label, test, test_label = get_data()

dt_start = time.time()
tree = decision_tree(train, train_label)
dt_end = time.time()

shrubs, max_time = neural_shrubs(tree, train, train_label)

Loading  ./sensIT_train.csv  dataset ...

Dataset loaded successfully


Loading  ./sensIT_test.csv  dataset ...

Dataset loaded successfully


(3289, 49) (3289, 3)
3


/home/shashwati/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(30, input_dim=49, activation="relu", kernel_initializer="uniform")`
/home/shashwati/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="relu", kernel_initializer="uniform")`
/home/shashwati/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="softmax", kernel_initializer="uniform")`


Epoch 1/15
3289/3289 [==============================] - 0s 119us/step - loss: 1.0974 - acc: 0.4044
Epoch 2/15
3289/3289 [==============================] - 0s 13us/step - loss: 1.0944 - acc: 0.4099
Epoch 3/15
3289/3289 [==============================] - 0s 15us/step - loss: 1.0904 - acc: 0.4099
Epoch 4/15
3289/3289 [==============================] - 0s 13us/step - loss: 1.0845 - acc: 0.4099
Epoch 5/15
3289/3289 [==============================] - 0s 11us/step - loss: 1.0767 - acc: 0.4099
Epoch 6/15
3289/3289 [==============================] - 0s 13us/step - loss: 1.0664 - acc: 0.4099
Epoch 7/15
3289/3289 [==============================] - 0s 10us/step - loss: 1.0545 - acc: 0.4099
Epoch 8/15
3289/3289 [==============================] - 0s 12us/step - loss: 1.0436 - acc: 0.4099
Epoch 9/15
3289/3289 [==============================] - 0s 18us/step - loss: 1.0381 - acc: 0.4099
Epoch 10/15
3289/3289 [==============================] - 0s 14us/step - loss: 1.0367 - acc: 0.4099
Epoch 11/15
3289/3

3479/3479 [==============================] - 0s 11us/step - loss: 0.8459 - acc: 0.9825
Epoch 9/15
3479/3479 [==============================] - 0s 8us/step - loss: 0.7545 - acc: 0.9825
Epoch 10/15
3479/3479 [==============================] - 0s 9us/step - loss: 0.6503 - acc: 0.9825
Epoch 11/15
3479/3479 [==============================] - 0s 7us/step - loss: 0.5419 - acc: 0.9825
Epoch 12/15
3479/3479 [==============================] - 0s 9us/step - loss: 0.4419 - acc: 0.9825
Epoch 13/15
3479/3479 [==============================] - 0s 9us/step - loss: 0.3586 - acc: 0.9825
Epoch 14/15
3479/3479 [==============================] - 0s 10us/step - loss: 0.2932 - acc: 0.9825
Epoch 15/15
3479/3479 [==============================] - 0s 10us/step - loss: 0.2442 - acc: 0.9825
(500, 49) (500, 3)
3
Epoch 1/15
500/500 [==============================] - 0s 899us/step - loss: 1.0984 - acc: 0.5280
Epoch 2/15
500/500 [==============================] - 0s 24us/step - loss: 1.0978 - acc: 0.5260
Epoch 3/15
5

Epoch 1/15
520/520 [==============================] - 1s 1ms/step - loss: 1.0984 - acc: 0.7327
Epoch 2/15
520/520 [==============================] - 0s 30us/step - loss: 1.0956 - acc: 0.9096
Epoch 3/15
520/520 [==============================] - 0s 22us/step - loss: 1.0926 - acc: 0.9096
Epoch 4/15
520/520 [==============================] - 0s 23us/step - loss: 1.0895 - acc: 0.9096
Epoch 5/15
520/520 [==============================] - 0s 27us/step - loss: 1.0862 - acc: 0.9096
Epoch 6/15
520/520 [==============================] - 0s 26us/step - loss: 1.0827 - acc: 0.9096
Epoch 7/15
520/520 [==============================] - 0s 25us/step - loss: 1.0789 - acc: 0.9096
Epoch 8/15
520/520 [==============================] - 0s 23us/step - loss: 1.0747 - acc: 0.9096
Epoch 9/15
520/520 [==============================] - 0s 32us/step - loss: 1.0701 - acc: 0.9096
Epoch 10/15
520/520 [==============================] - 0s 27us/step - loss: 1.0649 - acc: 0.9096
Epoch 11/15
520/520 [===================

...

595/595 [==============================] - 0s 24us/step - loss: 1.0967 - acc: 0.3597
Epoch 13/15
595/595 [==============================] - 0s 28us/step - loss: 1.0964 - acc: 0.3597
Epoch 14/15
595/595 [==============================] - 0s 24us/step - loss: 1.0959 - acc: 0.3597
Epoch 15/15
595/595 [==============================] - 0s 26us/step - loss: 1.0954 - acc: 0.3597
(2167, 49) (2167, 3)
3
Epoch 1/15
2167/2167 [==============================] - 1s 634us/step - loss: 1.0975 - acc: 0.5565
Epoch 2/15
2167/2167 [==============================] - 0s 12us/step - loss: 1.0943 - acc: 0.5981
Epoch 3/15
2167/2167 [==============================] - 0s 13us/step - loss: 1.0907 - acc: 0.5981
Epoch 4/15
2167/2167 [==============================] - 0s 14us/step - loss: 1.0868 - acc: 0.5981
Epoch 5/15
2167/2167 [==============================] - 0s 23us/step - loss: 1.0821 - acc: 0.5981
Epoch 6/15
2167/2167 [==============================] - 0s 26us/step - loss: 1.0764 - acc: 0.5981
Epoch 7/15
2

752/752 [==============================] - 0s 16us/step - loss: 1.0959 - acc: 0.5239
Epoch 5/15
752/752 [==============================] - 0s 18us/step - loss: 1.0950 - acc: 0.5239
Epoch 6/15
752/752 [==============================] - 0s 18us/step - loss: 1.0940 - acc: 0.5239
Epoch 7/15
752/752 [==============================] - 0s 21us/step - loss: 1.0930 - acc: 0.5239
Epoch 8/15
752/752 [==============================] - 0s 15us/step - loss: 1.0918 - acc: 0.5239
Epoch 9/15
752/752 [==============================] - 0s 14us/step - loss: 1.0906 - acc: 0.5239
Epoch 10/15
752/752 [==============================] - 0s 17us/step - loss: 1.0892 - acc: 0.5239
Epoch 11/15
752/752 [==============================] - 0s 21us/step - loss: 1.0879 - acc: 0.5239
Epoch 12/15
752/752 [==============================] - 0s 17us/step - loss: 1.0862 - acc: 0.5239
Epoch 13/15
752/752 [==============================] - 0s 15us/step - loss: 1.0845 - acc: 0.5239
Epoch 14/15
752/752 [==========================

694/694 [==============================] - 0s 20us/step - loss: 1.0888 - acc: 0.5418
Epoch 13/15
694/694 [==============================] - 0s 16us/step - loss: 1.0877 - acc: 0.5418
Epoch 14/15
694/694 [==============================] - 0s 19us/step - loss: 1.0866 - acc: 0.5418
Epoch 15/15
694/694 [==============================] - 0s 21us/step - loss: 1.0855 - acc: 0.5418
(506, 49) (506, 3)
3
Epoch 1/15
506/506 [==============================] - 1s 3ms/step - loss: 1.0985 - acc: 0.4269
Epoch 2/15
506/506 [==============================] - 0s 26us/step - loss: 1.0979 - acc: 0.4368
Epoch 3/15
506/506 [==============================] - 0s 16us/step - loss: 1.0974 - acc: 0.4170
Epoch 4/15
506/506 [==============================] - 0s 20us/step - loss: 1.0968 - acc: 0.4170
Epoch 5/15
506/506 [==============================] - 0s 25us/step - loss: 1.0963 - acc: 0.4170
Epoch 6/15
506/506 [==============================] - 0s 31us/step - loss: 1.0958 - acc: 0.4170
Epoch 7/15
506/506 [========

559/559 [==============================] - 0s 22us/step - loss: 1.0931 - acc: 0.6637
Epoch 6/15
559/559 [==============================] - 0s 21us/step - loss: 1.0918 - acc: 0.6637
Epoch 7/15
559/559 [==============================] - 0s 18us/step - loss: 1.0903 - acc: 0.6637
Epoch 8/15
559/559 [==============================] - 0s 28us/step - loss: 1.0889 - acc: 0.6637
Epoch 9/15
559/559 [==============================] - 0s 23us/step - loss: 1.0874 - acc: 0.6637
Epoch 10/15
559/559 [==============================] - 0s 25us/step - loss: 1.0859 - acc: 0.6637
Epoch 11/15
559/559 [==============================] - 0s 21us/step - loss: 1.0843 - acc: 0.6637
Epoch 12/15
559/559 [==============================] - 0s 17us/step - loss: 1.0826 - acc: 0.6637
Epoch 13/15
559/559 [==============================] - 0s 27us/step - loss: 1.0809 - acc: 0.6637
Epoch 14/15
559/559 [==============================] - 0s 33us/step - loss: 1.0791 - acc: 0.6637
Epoch 15/15
559/559 [=========================

515/515 [==============================] - 0s 23us/step - loss: 1.0820 - acc: 0.4757
Epoch 14/15
515/515 [==============================] - 0s 23us/step - loss: 1.0799 - acc: 0.4757
Epoch 15/15
515/515 [==============================] - 0s 34us/step - loss: 1.0778 - acc: 0.4757
(579, 49) (579, 3)
3
Epoch 1/15
579/579 [==============================] - 2s 3ms/step - loss: 1.0985 - acc: 0.4404
Epoch 2/15
579/579 [==============================] - 0s 25us/step - loss: 1.0973 - acc: 0.4421
Epoch 3/15
579/579 [==============================] - 0s 22us/step - loss: 1.0961 - acc: 0.4421
Epoch 4/15
579/579 [==============================] - 0s 28us/step - loss: 1.0949 - acc: 0.4404
Epoch 5/15
579/579 [==============================] - 0s 29us/step - loss: 1.0936 - acc: 0.4404
Epoch 6/15
579/579 [==============================] - 0s 23us/step - loss: 1.0922 - acc: 0.4404
Epoch 7/15
579/579 [==============================] - 0s 20us/step - loss: 1.0907 - acc: 0.4404
Epoch 8/15
579/579 [=========

In [10]:
def neural_shrub_predict(tree, nn_model, test, label):
    label_test = np.array(label)
    test = np.array(test)
    
    #row - actual; col - pred
    confusion_matrix = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
    correct = 0
    
    for i in range(len(test)):
        x = test[i]
        pred_leaf = tree.apply([x])
        x = np.array([x])
        nn_model_class = nn_model[pred_leaf[0]]
        pred = np.argmax(nn_model_class.predict(x))+1

        confusion_matrix[label[i]-1][pred-1] = confusion_matrix[label[i]-1][pred-1] + 1
        if pred == label[i]: correct = correct + 1

    acc_score = correct/len(test)
    
    return confusion_matrix, acc_score

In [11]:
# Predicting
cm, acc_score = neural_shrub_predict(tree, shrubs, test, test_label)
print("Confusion Matrix:\n\n", cm)

Confusion Matrix:

 [[3030 1338  235]
 [1278 3167  865]
 [ 833 1669 7290]]


In [12]:
def metrics(cm, cls, size):
    cm = np.array(cm)
    tp = cm[cls][cls]
    fp = sum(cm[x, cls] for x in range(3))-cm[cls][cls]
    fn = sum(cm[cls, x] for x in range(3))-cm[cls][cls]
    tn = size - tp - fp - fn
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    fmeasure = 2*(precision*recall)/(precision + recall)
    accuracy = (tp + tn)/size
    
    return precision, recall, fmeasure, accuracy

In [13]:
# Class 1
precision0, recall0, f0, acc0 = metrics(cm, 0, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 1: ", round(precision0, 3), "  ", round(recall0, 3), \
      " ", round(f0, 3), "   ", round(acc0,3))

        Precision Recall F-measure Accuracy
Class 1:  0.589    0.658   0.622     0.813


In [14]:
# Class 2
precision1, recall1, f1, acc1 = metrics(cm, 1, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 2: ", round(precision1, 3), "  ", round(recall1, 3), \
      " ", round(f1, 3), "   ", round(acc1,3))

        Precision Recall F-measure Accuracy
Class 2:  0.513    0.596   0.552     0.739


In [15]:
# Class 3
precision2, recall2, f2, acc2 = metrics(cm, 2, len(test))
print("        Precision Recall F-measure Accuracy")
print("Class 3: ", round(precision2, 3), "  ", round(recall2, 3), \
      " ", round(f2, 3), "   ", round(acc2,3))

        Precision Recall F-measure Accuracy
Class 3:  0.869    0.744   0.802     0.817


In [16]:
avg_p = (precision0 + precision1 + precision2)/3.0
avg_r = (recall0 + recall1 + recall2) / 3.0
avg_f = (f0 + f1 + f2) / 3.0
avg_a = (acc0 + acc1 + acc2)/ 3.0
print("        Precision Recall F-measure Accuracy")
print("Average: ", round(avg_p, 3), "  ", round(avg_r, 3), \
      " ", round(avg_f, 3), "   ", round(avg_a,3))

        Precision Recall F-measure Accuracy
Average:  0.657    0.666   0.658     0.79


In [17]:
# Number of instances correctly classified
print("Accuracy_score: ", round(acc_score, 4))
total_time_taken = dt_end - dt_start + max_time
print("Training Time: %s secs" % round(total_time_taken, 3))

Accuracy_score:  0.6844
Training Time: 5.764 secs
